# Table 6: Accuracy on 32x32 flowpic when enlargin training set (w/o Dropout)

[:simple-jupyter: :material-download:](../../paper_tables_and_figures/table6_simclr_larger_trainset/table6_simclr_larger_trainset.ipynb)

In [1]:
import os
import pathlib
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.stats.api as sms
import yaml
from aim import Repo

In [3]:
def compute_confidence_intervals(array, alpha=0.05):
    array = np.array(array)
    low, high = sms.DescrStatsW(array).tconfint_mean(alpha)
    mean = array.mean()
    ci = high - mean
    return ci

In [ ]:
campaign_folder = Path(
    "./campaigns/ucdavis-icdm19/larger-trainset/augmentation-at-loading"
)

repo = Repo.from_path(str(campaign_folder))

ct_df = pd.DataFrame(
    columns=[
        "hash_id",
        "contrastive_learning_seed",
        "finetune_seed",
        "suppress_dropout",
        "test_set",
        "test_acc",
    ]
)
sup_df = pd.DataFrame(
    columns=["hash_id", "aug_name", "seed", "suppress_dropout", "test_set", "test_acc"]
)


ct_nodropout_hashes = []
sup_nodropout_hashes = []
for hash_id in list(repo._all_run_hashes()):
    path = campaign_folder / "artifacts" / hash_id / "params.yml"
    if os.path.isfile(path):
        conf = yaml.safe_load(path.read_text())

        suppress_dropout = conf["suppress_dropout"]

        human_acc = (
            pd.read_csv(
                campaign_folder / "artifacts" / hash_id / "test-human_class_rep.csv"
            )
            .iloc[5]
            .values[1]
        )

        script_acc = (
            pd.read_csv(
                campaign_folder / "artifacts" / hash_id / "test-script_class_rep.csv"
            )
            .iloc[5]
            .values[1]
        )

        if conf["aim_experiment_name"] == "contrastive-learning-and-finetune":
            if suppress_dropout:
                ct_nodropout_hashes.append(hash_id)

                contrastive_learning_seed = conf["contrastive_learning_seed"]
                finetune_seed = conf["finetune_seed"]

                new_row_human = {
                    "hash_id": hash_id,
                    "contrastive_learning_seed": contrastive_learning_seed,
                    "finetune_seed": finetune_seed,
                    "suppress_dropout": suppress_dropout,
                    "test_set": "human",
                    "test_acc": human_acc * 100,
                }
                new_row_script = {
                    "hash_id": hash_id,
                    "contrastive_learning_seed": contrastive_learning_seed,
                    "finetune_seed": finetune_seed,
                    "suppress_dropout": suppress_dropout,
                    "test_set": "script",
                    "test_acc": script_acc * 100,
                }

                ct_df.loc[len(ct_df)] = new_row_human
                ct_df.loc[len(ct_df)] = new_row_script

        elif conf["aim_experiment_name"] == "augmentation-at-loading":
            if suppress_dropout:
                sup_nodropout_hashes.append(hash_id)

                seed = conf["seed"]

                aug_name = conf["aug_name"]

                new_row_human = {
                    "hash_id": hash_id,
                    "aug_name": aug_name,
                    "seed": seed,
                    "suppress_dropout": suppress_dropout,
                    "test_set": "human",
                    "test_acc": human_acc * 100,
                }
                new_row_script = {
                    "hash_id": hash_id,
                    "aug_name": aug_name,
                    "seed": seed,
                    "suppress_dropout": suppress_dropout,
                    "test_set": "script",
                    "test_acc": script_acc * 100,
                }

                sup_df.loc[len(sup_df)] = new_row_human
                sup_df.loc[len(sup_df)] = new_row_script

        else:  # aim runs rm
            print(1, hash_id)

    else:  # aim runs rm
        print(2, hash_id)

In [51]:
sup_df_merged = sup_df.groupby(
    [
        "suppress_dropout",
        "aug_name",
        "test_set",
    ]
).agg({"test_acc": ["mean", compute_confidence_intervals]})
sup_df_merged = sup_df_merged.rename(columns={"compute_confidence_intervals": "ci95"})
sup_df_merged = sup_df_merged.droplevel(0, axis=1)

sup_df_merged = sup_df_merged.droplevel(0).reset_index()

In [52]:
sup_df_merged = sup_df_merged.pivot(
    index="aug_name", columns="test_set", values=["mean", "ci95"]
)
sup_df_merged = sup_df_merged[
    [("mean", "script"), ("ci95", "script"), ("mean", "human"), ("ci95", "human")]
]
sup_df_merged.columns = pd.MultiIndex.from_tuples(
    tpl[::-1] for tpl in sup_df_merged.columns
)

In [54]:
AUGMENTATIONS_ORDER = [
    "noaug",
    "rotate",
    "horizontalflip",
    "colorjitter",
    "packetloss",
    "timeshift",
    "changertt",
]

RENAME = {
    "noaug": "No augmentation",
    "changertt": "Change rtt",
    "horizontalflip": "Horizontal flip",
    "colorjitter": "Color jitter",
    "packetloss": "Packet loss",
    "rotate": "Rotate",
    "timeshift": "Time shift",
}

print("=== supervised augmentation (at loading) ===")
sup_df_merged.loc[AUGMENTATIONS_ORDER].rename(RENAME).round(2)

=== supervised augmentation (at loading) ===


script        human      
                  mean  ci95   mean  ci95
aug_name                                 
No augmentation  98.40  0.21  73.31  1.01
Rotate           98.53  0.13  74.16  1.22
Horizontal flip  98.20  0.18  74.52  1.15
Color jitter     98.43  0.32  72.71  0.64
Packet loss      98.70  0.19  72.89  0.59
Time shift       98.70  0.26  72.71  0.97
Change rtt       98.53  0.13  71.45  1.04

In [30]:
ct_df_merged = ct_df.groupby(["suppress_dropout", "test_set"]).agg(
    {"test_acc": ["mean", compute_confidence_intervals]}
)
ct_df_merged = ct_df_merged.rename(columns={"compute_confidence_intervals": "ci95"})
ct_df_merged = ct_df_merged.droplevel(0, axis=1).droplevel(0)

print("=== using simclr and finetuning ===")
ct_df_merged.loc[["script", "human"]].round(2)

=== using simclr and finetuning ===


,mean,ci95
test_set,,
script,94.10,0.48
human,80.61,2.96
